<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Judy/For%20Lilian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nstock = 3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cupy
import jax
import jax.numpy as jnp
from jax import random
from jax import jit
import numpy as np
import pandas as pd
from torch.utils.dlpack import from_dlpack

def Brownian_motion(key, initial_stocks, numsteps, drift, cov, T):
    stocks_init = jnp.zeros((numsteps + 1, initial_stocks.shape[0]))
    stocks_init = jax.ops.index_update(stocks_init,   # jax.ops.index_update(x, idx, y) <-> Pure equivalent of x[idx] = y
                            jax.ops.index[0],         # initialization of stock prices
                            initial_stocks)
    noise = jax.random.multivariate_normal(key,  jnp.array([0]*initial_stocks.shape[0]), cov, (numsteps+1,)) # noise must have mean 0
    sigma = jnp.diag(cov) ** 0.5
    dt = T / numsteps
    def time_step(t, val):
        dx = jnp.exp((drift - sigma ** 2. / 2.) * dt + jnp.sqrt(dt) * noise[t,:])
        val = jax.ops.index_update(val,
                            jax.ops.index[t],
                            val[t-1] * dx)
        return val
    return jax.lax.fori_loop(1, numsteps+1, time_step, stocks_init)[1:] # jax.lax.fori_loop(lower, upper, body_fun, init_val)

def optionvalueavg(key, initial_stocks, numsteps, drift, r, cov, K, B, T): # down-and-out call
    return jnp.mean(jnp.maximum((1 - jnp.any(jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2) < B, axis=1).astype(int))* 
                                (jnp.mean(batch_simple(keys, initial_stocks, numsteps, drift, cov, T), axis=2))[:,-1]-K, 0) *
                    jnp.exp(-r[0] * T))

goptionvalueavg = jax.grad(optionvalueavg, argnums=1)

#################################################################### Adjust all parameters here (not inside class)
numstocks = nstock
numsteps = 50
numpaths = 2000000

rng = jax.random.PRNGKey(np.random.randint(10000))
rng, key = jax.random.split(rng)
keys = jax.random.split(key, numpaths)

S1_range = jnp.linspace(0.75, 1.25, 6)[4:6]
S2_range = jnp.linspace(0.75, 1.25, 6)
S3_range = jnp.linspace(0.75, 1.25, 6)
K_range = jnp.linspace(0.75, 1.25, 5)
B_range = jnp.linspace(0.5, 1.0, 6)
sigma_range = jnp.linspace(0.15, 0.45, 3)
r_range = jnp.linspace(0.01, 0.04, 3)
T = 1.0

fast_simple = jax.jit(Brownian_motion, static_argnums=2)
batch_simple = jax.vmap(fast_simple, in_axes=(0, None, None, None, None, None))
####################################################################

call = []
count = 0

for S1 in S1_range:
  for S2 in S2_range:
    for S3 in S3_range:
      for K in K_range:
        for B in B_range:
          for r in r_range:
            for sigma in sigma_range:
              
              initial_stocks = jnp.array([S1, S2, S3]) # must be float
              r_tmp = jnp.array([r]*numstocks)
              drift = r_tmp
              cov = jnp.identity(numstocks)*sigma*sigma
              
              Knock_Out_Call_price = optionvalueavg(key, initial_stocks, numsteps, drift, r_tmp, cov, K, B, T)
              Deltas = goptionvalueavg(keys, initial_stocks, numsteps, drift, r_tmp, cov, K, B, T)
              call.append([T, K, B, S1, sigma, r, r,
                          T, K, B, S2, sigma, r, r,
                          T, K, B, S3, sigma, r, r, Knock_Out_Call_price] + list(Deltas)) #T, K, S, sigma, mu, r, price, delta
                          
              count += 1
              print(count)

Thedataset_3stock = pd.DataFrame(call)
Thedataset_3stock.to_csv(f'/content/drive/MyDrive/AFP Project/Lilian/Knock_Out_Call_3stock_MC_Datset.csv', index=False, header=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


KeyboardInterrupt: ignored

In [ ]:
Thedataset_3stock = pd.DataFrame(call)
Thedataset_3stock

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,1.0,0.75,0.5,0.95,0.15,0.010,0.010,1.0,0.75,0.5,0.75,0.15,0.010,0.010,1.0,0.75,0.5,0.75,0.15,0.010,0.010,0.07889941,0.291181,0.289860,0.289824
1,1.0,0.75,0.5,0.95,0.30,0.010,0.010,1.0,0.75,0.5,0.75,0.30,0.010,0.010,1.0,0.75,0.5,0.75,0.30,0.010,0.010,0.099639826,0.247038,0.243098,0.243069
2,1.0,0.75,0.5,0.95,0.45,0.010,0.010,1.0,0.75,0.5,0.75,0.45,0.010,0.010,1.0,0.75,0.5,0.75,0.45,0.010,0.010,0.1255407,0.231955,0.226037,0.225984
3,1.0,0.75,0.5,0.95,0.15,0.025,0.025,1.0,0.75,0.5,0.75,0.15,0.025,0.025,1.0,0.75,0.5,0.75,0.15,0.025,0.025,0.08852259,0.301916,0.300854,0.300811
4,1.0,0.75,0.5,0.95,0.30,0.025,0.025,1.0,0.75,0.5,0.75,0.30,0.025,0.025,1.0,0.75,0.5,0.75,0.30,0.025,0.025,0.10724648,0.255975,0.252295,0.252257
5,1.0,0.75,0.5,0.95,0.45,0.025,0.025,1.0,0.75,0.5,0.75,0.45,0.025,0.025,1.0,0.75,0.5,0.75,0.45,0.025,0.025,0.13212144,0.238295,0.232578,0.232536
6,1.0,0.75,0.5,0.95,0.15,0.040,0.040,1.0,0.75,0.5,0.75,0.15,0.040,0.040,1.0,0.75,0.5,0.75,0.15,0.040,0.040,0.09835694,0.310457,0.309624,0.309579
7,1.0,0.75,0.5,0.95,0.30,0.040,0.040,1.0,0.75,0.5,0.75,0.30,0.040,0.040,1.0,0.75,0.5,0.75,0.30,0.040,0.040,0.11506154,0.264336,0.260942,0.260886
8,1.0,0.75,0.5,0.95,0.45,0.040,0.040,1.0,0.75,0.5,0.75,0.45,0.040,0.040,1.0,0.75,0.5,0.75,0.45,0.040,0.040,0.13883741,0.244505,0.239001,0.238955
9,1.0,0.75,0.6,0.95,0.15,0.010,0.010,1.0,0.75,0.6,0.75,0.15,0.010,0.010,1.0,0.75,0.6,0.75,0.15,0.010,0.010,0.07889941,0.291181,0.289860,0.289824


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#save to csv
Thedataset_3stock.to_csv(f'/content/drive/MyDrive/AFP/Save_Models/Knock_Out_Call_{str(nstock)}stock_MC_Datset.csv', index=False, header=False)

In [ ]:
# read csv
import pandas as pd

Thedataset_3stock = pd.read_csv(f'/content/drive/MyDrive/AFP/Save_Models/Knock_Out_Call_{str(nstock)}stock_MC_Datset.csv', header=None)
Thedataset_3stock